In [55]:
from gurobipy import *
from math import *
from random import randint
import random
import sys
import random
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from openpyxl.workbook import Workbook
import copy

In [56]:
################################################################
## INPUT FILES
################################################################
instances = range(1,101) # instance number
network = '60' #number of links or name of the network

network_file = 'Network' + network + '.xlsx'
inst_file = 'Instance26week.xlsx'
arrivals = 'admit.xlsx'
tot_days = 26*7

In [57]:
################################################################
## READ IN DATA FROM FILES
################################################################
df_I = pd.read_excel(inst_file, sheet_name='ICU') # data frame of ICU
df_J = pd.read_excel(inst_file, sheet_name='DSU') # data frame of DSU
df_G = pd.read_excel(network_file, sheet_name='Network') # data frame network
df_A = pd.read_excel(arrivals,index_col=0) # data frame arrivals

I = df_I['ICU name'].tolist() # ICU list
J = df_J['DSU name'].tolist() # DSU list

C_bar = df_I.set_index('ICU name')['Capacity'].to_dict() # total ICU capacity
P_init = df_I.set_index('ICU name')['O1'].to_dict() # initial patient occupancy at ICU

D_bar = df_J.set_index('DSU name')['Capacity'].to_dict() # total DSU capacity
D_prime_bar = df_J.set_index('DSU name')['Variable capacity'].to_dict() # total DSU flexible capacity
Q_init = df_J.set_index('DSU name')['O1'].to_dict() # initial occupancy at DSU
Q_prime_init = df_J.set_index('DSU name')['O1_vc'].to_dict()

Ji = {} # list of feasible DSUs for each ICU
for i in I:
    df_G_i = df_G[df_G['ICU'] == i]
    Ji[i] = df_G_i['DSU'].tolist()

Ij = {} # list of feasible ICUs for each DSU
for j in J:
    df_G_j = df_G[df_G['DSU'] == j]
    Ij[j] = df_G_j['ICU'].tolist()
#print J, len(J)

In [58]:
tempJ = []
for j in J:
    if len(Ij[j]) > 0:
        tempJ.append(j)
J = tempJ
#print J, len(J)

In [59]:
################################################################
## PARAMETERS
################################################################

b = {} # Price for flexible capacity
for j in J:
    b[j] = 0.001 # for simple example
    
l = {} # Initial guess for lambda prices out of ICUs
for i in I:
    l[i] = 0.1
    
iter = 100 # Loop iterations
step = 0.001

In [60]:
def U(a,y):
    ''' Utility function
        '''
    return 1-exp(-a*y)

def ICU(a,C,lam):
    ''' A function that solves the ICU problem and returns the
        value of y
        '''
    y = {}
    y0 = {}
    y1 = {}
    y2 = {}
    y3 = {}
    for i in I:
        if lam[i] <= 0:
            y[i] = C[i]
        else:
            y0[i] = 0
            y3[i] = C[i]
            stat_point = (-1/float(a[i]))*log(lam[i]/float(a[i]))
            y1[i] = floor(stat_point)
            y2[i] = ceil(stat_point)
            Obj = {}
            Obj[y0[i]] = U(a[i], y0[i]) - lam[i]*y0[i]
            Obj[y3[i]] = U(a[i], y3[i]) - lam[i]*y3[i]
            if y1[i] >= 0 and y1[i] <= C[i]:
                Obj[y1[i]] = U(a[i], y1[i]) - lam[i]*y1[i]
            if y2[i] >= 0 and y2[i] <= C[i]:
                Obj[y2[i]] = U(a[i], y2[i]) - lam[i]*y2[i]
            y[i] = max(Obj.iteritems(), key=operator.itemgetter(1))[0]
    return y

def dualobj(a,x,z,y,lam):
    ''' A function calculating the dual objective value for a given
    price and solutions x, z, y
    '''
    v = sum(U(a[i],y[i]) - lam[i]*y[i] for i in I) + sum(sum(lam[i]*x[i,j] + (lam[i]-b[j])*z[i,j] for j in Ji[i]) for i in I)
    return v

def netLP(C,D,D_prime,lam,nu):
    ''' A function that solves the network LP for a given price vector p and returns the
        values of x and z'''
    m = Model()
    x = {}
    z = {}
    d = {}
    for i in I:
        for j in Ji[i]:
            x[i,j] = m.addVar(lb = 0, ub = C[i], vtype = GRB.CONTINUOUS, name = "x[%s,%s]" % (i,j))
            z[i,j] = m.addVar(lb = 0, ub = C[i], vtype = GRB.CONTINUOUS, name = "z[%s,%s]" % (i,j))
    
    m.update()
    m.setObjective(sum(sum(lam[i]*x[i,j] + (lam[i]-b[j])*z[i,j] for j in Ji[i]) for i in I), GRB.MAXIMIZE)
    for j in J:
        m.addConstr(sum(x[i,j] for i in Ij[j]) <= D[j])
        m.addConstr(sum(z[i,j] for i in Ij[j]) <= D_prime[j])

    m.addConstr(sum(sum(z[i,j] for j in Ji[i]) for i in I) <= nu)

    m.optimize()
    xs = {}
    zs = {}
    for i in I:
        for j in Ji[i]:
            xs[i,j] = m.getVarByName('x[%s,%s]' % (i,j)).x
            zs[i,j] = m.getVarByName('z[%s,%s]' % (i,j)).x
    return xs, zs

In [61]:
def findLambda(a,nu,C,D,D_prime):
    # Subgradient algorithm 
    grad = {}
    pr = {}
    xk = {}
    zk = {}
    yk = {}
    objk = {}

    pr[0] = l #initial guess
    xk[0], zk[0] = netLP(C,D,D_prime,pr[0],nu)
    yk[0] = ICU(a,C,pr[0])

    grad[0] = {}
    for i in I:
        grad[0][i] = sum((xk[0][i,j] + zk[0][i,j]) for j in Ji[i]) - yk[0][i]

    objk[0] = dualobj(a,xk[0],zk[0],yk[0],pr[0])
    q_best = objk[0]

    iter_tobest = 0

    p_best = pr[0]
    x_best = xk[0]
    z_best = zk[0]
    y_best = yk[0]


    for k in range(1,iter+1):

        # Candidate price, x, z, and y
        p = {}
        for i in I:
            p[i] = pr[k-1][i] - (step/sqrt(k))*grad[k-1][i]

        x, z = netLP(C,D,D_prime,p,nu)
        y = ICU(a,C,p)
        g = {}
        for i in I:
            g[i] = sum((x[i,j] + z[i,j]) for j in Ji[i]) - y[i]

        q = dualobj(a,x,z,y,p)

        pr[k] = p
        xk[k] = x
        zk[k] = z
        yk[k] = y
        grad[k] = g
        objk[k] = q

        if q <= q_best: # if improvement, update
            q_best = objk[k]
            p_best = pr[k]
            x_best = xk[k]
            z_best = zk[k]
            y_best = yk[k]
            iter_tobest = k
    
    return p_best



In [62]:
D = {}
for j in J:
    D[j] = D_bar[j]

priority = {}
for j in J:
    priority[j] = len(Ij[j])/float(D[j])
    
print(priority)
test = max(priority, key=priority.get)
print(test)

{'8D': 0.11764705882352941, '5D': 0.14285714285714285, '10D': 0.1, '11N': 0.1, '12S': 0.23809523809523808, '7F': 0.15, '4D': 0.047619047619047616, '9G': 0.12903225806451613, '12N': 0.0967741935483871, '5G': 0.125, 'R7SN': 0.07142857142857142, '7D': 0.075, '10S': 0.26666666666666666, '6D': 0.13953488372093023, '3E': 0.05263157894736842, '12D': 0.14285714285714285, '8G': 0.058823529411764705, '10G': 0.1, '10N': 0.2222222222222222, '9D': 0.03389830508474576, '8W': 0.038461538461538464, '8N': 0.1}
10S


In [63]:
def policyNEW(a,nu,C,D,D_prime):
    # Staff variable capacity beds in order of (Degree j)/(Available beds j) from highest to lowest
    Zvar = {}
    for j in J:
        Zvar[j] = 0
    
    candidate_J = []
    for j in J:
        if D_prime[j] > 0:
            candidate_J.append(j)
            
    priority = {} # initialize priorities for each DSU
    for j in candidate_J:
        if D[j] > 0:
            priority[j] = len(Ij[j])/float(D[j])
        else:
            priority[j] = 0
        
    avail_var = copy.deepcopy(D_prime)
    
    while nu > 0 and len(candidate_J) > 0:
        choice = max(priority, key=priority.get)
        j = choice
        Zvar[j] = Zvar[j] + 1
        nu = nu - 1
        avail_var[j] = avail_var[j] - 1
        # update candidate list of DSU with available var cap beds
        candidate_J = []
        for j in J:
            if avail_var[j] > 0:
                candidate_J.append(j)
        # update priority list
        if len(candidate_J) > 0:
            for j in candidate_J:
                if (D[j]+Zvar[j]) > 0:
                    priority[j] = len(Ij[j])/float(D[j]+Zvar[j])
                else:
                    priority[j] = 0
            
    # Allocate patients 
    sorted_I = sorted(I, key=lambda x: a[x])
    sorted_I.reverse()
    ready = copy.deepcopy(C)
    avail_bas = copy.deepcopy(D)
    avail_var = copy.deepcopy(Zvar)
    DSU = {}

    # ALLOCATE TO BASE CAPACITY
    for i in I: # for each ICU, maintain a list of DSU with available base capacity
        DSU[i] = []
        for j in Ji[i]:
            if avail_bas[j] > 0:
                DSU[i].append(j)
    x = {}
    for i in I:
        for j in Ji[i]:
            x[i,j] = 0

    I_priority = [] # sorted list of ICU by expected arrivals a[i] starting from highest
    for i in sorted_I: # an ICU is added to the list only if it has ready patients
        if ready[i] > 0 and len(DSU[i]) > 0: # and some of its DSUs have capacity
            I_priority.append(i)
    
    while len(I_priority) > 0:
        i = I_priority[0] # pick the highest priority ICU
        J_priority = {} # initialize DSU priority list
        for j in DSU[i]:
            if avail_bas[j] > 0:
                J_priority[j] = len(Ij[j])/float(avail_bas[j])
        choice = min(J_priority, key=J_priority.get)
        j = choice
        
        x[i,j] = x[i,j] + 1
        ready[i] = ready[i] - 1   
        avail_bas[j] = avail_bas[j] - 1

        for k in I: # update the DSU list of each ICU based on capacity
            DSU[k] = []
            for l in Ji[k]:
                if avail_bas[l] > 0:
                    DSU[k].append(l)

        I_priority2 = [] # update ICU priority list  
        ind_i = I_priority.index(i)+1
        for k in I_priority[ind_i:]:
            if ready[k] > 0 and len(DSU[k]) > 0: 
                I_priority2.append(k)
        if ready[i] > 0 and len(DSU[i]) > 0: 
            I_priority2.append(i)
        I_priority = [] 
        I_priority = copy.deepcopy(I_priority2)

    # ALLOCATE TO VAR CAPACITY
    for i in I: # for each ICU, maintain a list of DSU with available var capacity
        DSU[i] = []
        for j in Ji[i]:
            if avail_var[j] > 0:
                DSU[i].append(j)
    z = {}
    for i in I:
        for j in Ji[i]:
            z[i,j] = 0

    I_priority = [] # sorted list of ICU by expected arrivals a[i] starting from highest
    for i in sorted_I: # an ICU is added to the list only if it has ready patients
        if ready[i] > 0 and len(DSU[i]) > 0: # and some of its DSUs have capacity
            I_priority.append(i)

    while len(I_priority) > 0:
        i = I_priority[0] # pick the highest priority ICU
        J_priority = {} # initialize DSU priority list
        for j in DSU[i]:
            if avail_var[j] > 0:
                J_priority[j] = len(Ij[j])/float(avail_var[j])
        choice = min(J_priority, key=J_priority.get)
        
        z[i,j] = z[i,j] + 1
        ready[i] = ready[i] - 1   
        avail_var[j] = avail_var[j] - 1

        for k in I: # update the DSU list of each ICU based on capacity
            DSU[k] = []
            for l in Ji[k]:
                if avail_var[l] > 0:
                    DSU[k].append(l)

        I_priority2 = [] # update ICU priority list  
        ind_i = I_priority.index(i)+1
        for k in I_priority[ind_i:]:
            if ready[k] > 0 and len(DSU[k]) > 0: 
                I_priority2.append(k)
        if ready[i] > 0 and len(DSU[i]) > 0: 
            I_priority2.append(i)
        I_priority = [] 
        I_priority = copy.deepcopy(I_priority2)      
    
            
    return x, z

In [64]:
C_inst = {}
A_icu = {}
A_dsu = {}
Disch = {}
Disch_vc = {}
x = {}
z = {}

for day in range(1,tot_days+1):
    C_inst[day] = df_I.set_index('ICU name')['C'+str(day)].to_dict()
    A_icu[day] = df_I.set_index('ICU name')['A'+str(day)].to_dict()
    A_dsu[day] = df_J.set_index('DSU name')['A'+str(day)].to_dict()
    Disch[day] = df_J.set_index('DSU name')['C'+str(day)].to_dict()
    Disch_vc[day] = df_J.set_index('DSU name')['C_vc'+str(day)].to_dict()
    
nu_max = 100

In [65]:
################################################################
## TRANSITION FUNCTION
################################################################
def trans_func(day, P0, Q0, Q_prime0, nu0):
    '''A function to determine the transitions from one state to the next
        '''
    # Read transfer requests and new arrivals for the given day
    d = day%7
    if d == 0:
        d = 7
    a = {}
    for i in I:
        a[i] = df_A.loc[i, d]/10.0

    C = {}
    for i in I:
        C[i] = int(min(C_inst[day][i], P0[i]))    
    D = {}
    D_prime = {}
    for j in J:
        D[j] = D_bar[j]-Q0[j]
        D_prime[j] = D_prime_bar[j]-Q_prime0[j]
    #lam = findLambda(a,nu0,C,D,D_prime)
    
    # 1) Make transfer decisions given the state
    x, z = policyNEW(a,nu0,C,D,D_prime)
    
    # 2) Update the ICU populations by removing transfers and admitting new arrivals
    P1 = {}
    lost_icu = {}
    for i in I:
        P1[i] = min(P0[i]-sum(x[i,j]+z[i,j] for j in Ji[i])+A_icu[day][i], C_bar[i])
        lost_icu[i] = max(0, P0[i]-sum(x[i,j]+z[i,j] for j in Ji[i])+A_icu[day][i] - C_bar[i])
        
    # 3) Update the DSU populations by
        # a) adding transfers b) discharging patients c) admitting new arrivals
    Q1 = {}
    Q1_prime = {}
    lost_dsu = {}

    for j in J:
        # a) add transfers 
        population = Q0[j] + sum(x[i,j] for i in Ij[j])
        population_vc = Q_prime0[j] + sum(z[i,j] for i in Ij[j])
        
        # b) discharge patients
        population = max(0, population - Disch[day][j])
        population_vc = max(0, population_vc - Disch_vc[day][j])
        
        # c) admit new arrivals
        Q1[j] = min(population + A_dsu[day][j], D_bar[j])
        lost_dsu[j] = max(0, population + A_dsu[day][j] - D_bar[j])
        Q1_prime[j] = population_vc
        
        
    nu1 = max(0,nu_max - sum(Q1_prime[j] for j in J))
    
    o = sum(U(a[i], sum(x[i,j] for j in Ji[i])) for i in I) - sum(b[j]*sum(z[i,j] for i in Ij[j]) for j in J)
    
    return P1, Q1, Q1_prime, nu1, x, z, lost_icu, lost_dsu, o


In [66]:
for inst in instances:

    #day = 1 # Monday
    P = {}
    P[0] = df_I.set_index('ICU name')['O1'].to_dict() 

    Q = {}
    Q[0] = df_J.set_index('DSU name')['O1'].to_dict() 

    Q_prime = {}
    Q_prime[0] = df_J.set_index('DSU name')['O1_vc'].to_dict() 

    nu = {}
    nu[0] = sum(Q_prime[0][j] for j in J) + int((sum(D_prime_bar[j] for j in J))/2)



    L_icu = {}
    L_dsu = {}
    o = {}

    for day in range(1,tot_days+1):
        P[day], Q[day], Q_prime[day], nu[day], x[day], z[day], L_icu[day], L_dsu[day], o[day] = trans_func(day, P[day-1], Q[day-1], Q_prime[day-1], nu[day-1])

    cols_icu = ['capacity'] + [str(d)+'_arrival_lost' for d in range(1,tot_days+1)] + [str(d)+'_obj' for d in range(1,tot_days+1)]
    out_df_icu = pd.DataFrame(columns=cols_icu, index=I)

    for i in I:
        out_df_icu.at[i, 'capacity'] = C_bar[i]
        for day in range(1,tot_days+1):       
            if A_icu[day][i] > 0:
                lost = 100*L_icu[day][i]/A_icu[day][i]
            else:
                lost = 0
            out_df_icu.at[i, str(day)+'_arrival_lost'] = lost
            obj = o[day]
            out_df_icu.at[i, str(day)+'_obj'] = obj

    out_df_icu.to_excel("policyNEW_icu_output_N"+str(network)+"_"+str(inst)+".xlsx")